In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/sales_transactions.csv')
df.head()

,Transaction_ID,Customer_ID,Region,Product_Category,Product_Name,Quantity,Unit_Price ($),Total_Sales ($),Transaction_Date,Payment_Method
0,10001,2861,East,Books,Biography,3,155.58,466.74,2024-09-09,PayPal
1,10002,3295,West,Toys,Action Figure,8,439.08,3512.64,2023-09-14,PayPal
2,10003,3131,North,Toys,Doll,8,276.15,2209.20,2024-08-17,Credit Card
3,10004,3096,North,Electronics,Laptop,2,291.73,583.46,2025-01-04,PayPal
4,10005,3639,South,Beauty,Perfume,7,332.55,2327.85,2024-03-09,PayPal


In [ ]:
# checking for null values
df.isna().sum() #no missing values
df.duplicated().sum() # No duplicate value

# standardizing column names
# first convert all to lower case, then replace $ with _USD and replace every
# with underscore
df.columns = df.columns.str.lower().str.replace(' ($)', '_(USD)').str.replace(' ', '_')

# checking the basic info of the df
df.describe()

# converting the date column to proper datetime type
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     10000 non-null  int64         
 1   customer_id        10000 non-null  int64         
 2   region             10000 non-null  object        
 3   product_category   10000 non-null  object        
 4   product_name       10000 non-null  object        
 5   quantity           10000 non-null  int64         
 6   unit_price_(usd)   10000 non-null  float64       
 7   total_sales_(usd)  10000 non-null  float64       
 8   transaction_date   10000 non-null  datetime64[ns]
 9   payment_method     10000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 781.4+ KB


In [19]:
total_revenue = df['total_sales_(usd)'].sum()
print(f"The total revenue obtained from all transactions was ${total_revenue}")

The total revenue obtained from all transactions was $13997042.07
